**Let the Fun Begin**
[Colab resources are not guaranteed](https://research.google.com/colaboratory/faq.html#:~:text=Colab\%20is\%20able\%20to\%20provide,other\%20factors\%20vary\%20over\%20time.). One can, however, [subscribe and increase resources]((https://colab.research.google.com/signup)).

<p>&nbsp;</p>


split_large_feacturecollection_to_blocks

In [ ]:
%who

Interactive namespace is empty.


In [ ]:
try:
    import shutup
except ImportError:
  !pip install shutup
  import shutup

shutup.please() # kill some of the messages

In [ ]:
"""
Print Local Time

colab runs on cloud. So, the time is not our local time.
This page is useful to determine how to do this.
"""
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/US/Pacific /etc/localtime # <--- The space after Pacific is needed
!date

Tue Dec  5 03:36:10 PM PST 2023


**geopandas and geemap must be installed every time!!!**

In [ ]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap not installed. Must be installed every tim to run this notebook. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

    print('geopandas not installed. Must be installed every time to run this notebook. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'google.colab'])

**Authenticate and import libraries**

We have to import the libraries we need. Moreover, we need to Authenticate, every single time!

In [ ]:
from datetime import datetime as DTDT
print (DTDT.now())

import pandas as pd
import numpy as np
import geopandas as gpd
import pickle
import folium, time, datetime, json, geemap, ee
# from datetime import date

# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

2023-12-05 15:36:12.525431
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6dWdQXa0M2TozCp_YpdHo-yNgRHHJVo8EY-zgEHkAcM&tc=tZrxUv6ChEUssUZNngL3kdJevYvfNQUOm1GCRyvSmNg&cc=MzAuow0yi22r9e7qFIDAkVpxsVSSjIpZIiVZBt2pmcw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXles4XkbGdM2VHbXzKQzLdNV_9WAOWj7P5UZx-PP0Oik7vu5KAbNUY

Successfully saved authorization token.


### **Mount Google Drive and import my Python modules**

Here we are importing the Python functions that are written by me and are needed; ```NASA core``` and ```NASA plot core```.

**Note:** These are on Google Drive now. Perhaps we can import them from GitHub.

In [ ]:
# Mount YOUR google drive in Colab
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,"/content/drive/My Drive/WSU_job/joel_pipeline//")
import NASA_core as nc
import NASA_plot_core as ncp
import GEE_Python_core as gpc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Please tell me where to look for the shapefile and other directories**

In [ ]:
drive_pre     = "/content/drive/My Drive/"
shp_path_base = drive_pre + "WSU_job/NASA_trends/shapefiles/"
out_dir_base  = drive_pre + "colab_outputs/"
Colab_NB_dir  = drive_pre + "Colab Notebooks/"

import os
os.chdir(Colab_NB_dir)
# !ls

In [ ]:
%%time

SF_name = "10_intersect_East_Irr_2008_2018_2cols"
# SF_name = "Grant_4Fields_poly_wCentroids"
shp_path = shp_path_base + SF_name + "/" + SF_name + ".shp"

"""
  we read our shapefile in to a geopandas dataframe using
  the geopandas.read_file method
  we'll make sure it's initiated in the EPSG 4326 CRS
"""
SF = gpd.read_file(shp_path, crs='EPSG:4326')

### for possible future use grab the data part of the shapefile
# SF_data = SF[["ID", "Acres", "county", "CropTyp", "DataSrc", "Irrigtn", "LstSrvD"]].copy()
SF_data = SF[["ID", "acreage"]].copy()
SF_data.drop_duplicates(inplace=True)
print (f"{SF_data.shape=}")

"""
   Drop extra useless columns. Saves space.
   Also, GEE behaves strangely. It had problem with Notes column before
"""
# SF = SF.drop(columns=["Notes", "TRS", "IntlSrD", "RtCrpTy", "Shp_Lng", "Shap_Ar","CropGrp","CropTyp","Acres","Irrigtn","LstSrvD","DataSrc","county","ExctAcr","cntrd_ln","cntrd_lt"])
# SF = SF.drop(columns=["CropTyp","Acres","Irrigtn", "LstSrvD","DataSrc","county","ExctAcr", "cntrd_ln","cntrd_lt"])
SF = SF.drop(columns=["acreage"])
SF.head(2)

SF_data.shape=(69271, 2)
CPU times: user 10.8 s, sys: 155 ms, total: 11 s
Wall time: 14.2 s


,ID,geometry
0,i1,"POLYGON ((-117.05541 47.74301, -117.05542 47.7..."
1,i2,"POLYGON ((-117.05540 47.74354, -117.05541 47.7..."


**Form Geographical Regions**

  - First, define a big region that covers Eastern Washington.
  - Convert shapefile to ```ee.featurecollection.FeatureCollection```.


In [ ]:
xmin, xmax = -125.0, -116.0
ymin, ymax = 45.0, 49.0

xmed = (xmin + xmax) / 2.0;
ymed = (ymin+ymax) / 2.0;

WA1 = ee.Geometry.Polygon([[xmin, ymin], [xmin, ymax], [xmed, ymax], [xmed, ymin], [xmin, ymin]]);
WA2 = ee.Geometry.Polygon([[xmed, ymin], [xmed, ymax], [xmax, ymax], [xmax, ymin], [xmed, ymin]]);
WA = [WA1, WA2];
big_rectangle = ee.FeatureCollection(WA);

In [ ]:
Map = geemap.Map(center=[ymed, xmed], zoom=7)
Map.addLayer(WA1, {'color': 'red'}, 'Western Half')
Map.addLayer(WA2, {'color': 'red'}, 'Eastern Half')
# Map.addLayer(curr_SF_geopandas, {'color': 'red'}, 'Fields')
Map

Map(center=[47.0, -120.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

**Define Parameters for Landsat 7**

We are interested in **1984-1993** and **1999-present**.

  - ```L4```: life span is ```16-July-1982``` to ```14-Dec-1993```.
  - ```L5```: life span is ```1-March-1984``` to ```5-June-2013```.
  - ```L7```: life span is ```15-April-1999``` to ```6-April-2022```.
  - ```L8```: life span is ```11-Feb-2013``` till now (```June 2023```).



  - **To Do- Merge Sentinel into ```extract_satellite_IC()``` function. (June 6 2023.)**

In [ ]:
"""
 sattelie_of_choice must be from
         'LANDSAT/LT04/C02/T1_L2' OR
         'LANDSAT/LT05/C02/T1_L2' OR
         'LANDSAT/LE07/C02/T1_L2' OR
         'LANDSAT/LC08/C02/T1_L2'
"""
sattelie_of_choice7 = 'LANDSAT/LE07/C02/T1_L2'
sattelie_of_choice5 = 'LANDSAT/LT05/C02/T1_L2'
sattelie_of_choice4 = 'LANDSAT/LT04/C02/T1_L2'

start_date_L7 = str(1999) + '-01-01'; # YYYY-MM-DD
end_date_L7   = str(2009) + '-01-01'; # YYYY-MM-DD

start_date_L5_early = str(1984) + '-01-01'; # YYYY-MM-DD
end_date_L5_early   = str(1994) + '-01-01'; # YYYY-MM-DD

start_date_L5_late = str(1999) + '-01-01'; # YYYY-MM-DD
end_date_L5_late   = str(2009) + '-01-01'; # YYYY-MM-DD

start_date_L4 = str(1984) + '-01-01'; # YYYY-MM-DD
end_date_L4   = str(1994) + '-01-01'; # YYYY-MM-DD

# start_date = "2017-01-01" # Date fromat for EE YYYY-MM-DD
# end_date = "2017-12-30"   # Date fromat for EE YYYY-MM-DD

print (DTDT.now())

2023-12-05 15:40:18.033271


In [ ]:
satellit_info = {"L4" : ['LANDSAT/LT04/C02/T1_L2', 1984, 1993],
                 "L5_early" : ['LANDSAT/LT05/C02/T1_L2', 1984, 1993],
                 "L5_late" : ['LANDSAT/LT05/C02/T1_L2', 1999, 2007],
                 "L7" : ['LANDSAT/LE07/C02/T1_L2', 1999, 2007]}

**Fetch data from GEE**


In [ ]:
%%time
block_size_ = 10
SFblocks = gpc.split_SF_to_blocks(SF, block_size=block_size_)

print (f"{len(SFblocks)=}")
print (f"{satellit_info.keys()=}")
print ()

len(SFblocks)=6928
satellit_info.keys()=dict_keys(['L4', 'L5_early', 'L5_late', 'L7'])

CPU times: user 14.9 s, sys: 86.2 ms, total: 14.9 s
Wall time: 15.1 s


In [ ]:
# print ("The size of [imageC.size().getInfo()] is [{:.0f}].".format(imageC.size().getInfo()))

In [ ]:
%%time
print (DTDT.now())
bad_counter=0
done_counter = 0
problems_L5_early = dict()
for a_sat_key in ["L5_early"]:
    a_sat_info = satellit_info[a_sat_key]
    satelliteChoice = a_sat_info[0]
    start_year = a_sat_info[1]
    end_year = a_sat_info[2]
    # for a_year in range(start_year, end_year+1):
    for a_year in reversed(range(start_year, end_year+1)):
        for a_SFblocks_key in list(SFblocks.keys()):
            curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key])
            st_date = str(a_year) + '-01-01'
            ed_date = str(a_year+1) + '-01-01'
            imageC = gpc.extract_satellite_IC(big_rectangle_featC = big_rectangle,
                                              start_date = st_date, end_date = ed_date,
                                              dataSource = satelliteChoice);

            reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
            """
              The reason I have try-except below is that
              it seems some years there is no data.
              Landsat 4 has been alive since 1982 but I am not getting
              data in 1984. I am not sure what is happening.
            """
            try:
              needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
              reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
              reduced = reduced[needed_columns]

              EVI = reduced[["ID", "EVI", "system_start_time"]]
              NDVI = reduced[["ID", "NDVI", "system_start_time"]]

              EVI.dropna(subset=["EVI"], inplace=True)
              NDVI.dropna(subset=["NDVI"], inplace=True)

              name_pref = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_)
              outfile_name = name_pref + "_EVI.csv"
              EVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

              outfile_name = name_pref + "_NDVI.csv"
              NDVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)
              done_counter += 1
              if done_counter%100==0:
                print (f"{done_counter=}")
              # print(f"{a_year = }, {a_SFblocks_key = }")
            except:
              bad_counter += 1
              if bad_counter%100==0:
                print (f"{bad_counter=}")
              problems_L5_early[a_sat_key + "_" + str(bad_counter)] = a_sat_key + "_year" + str(a_year) + "_SFblock" + str(a_SFblocks_key)
              file_Name = "problems_L5_early.pkl"

              f = open(out_dir_base + file_Name, "wb")
              pickle.dump(problems_L5_early, f)
              f.close() # close file
              print ("---------------------------------------------------------------------------------------------")
              print(f"ddd, {a_year = }, {a_SFblocks_key = }, {reduced.size().getInfo()=}")
              print ("---------------------------------------------------------------------------------------------")

print (DTDT.now())

2023-06-09 19:38:05.636025
done_counter=100
done_counter=200


In [ ]:
%%time
a_sat_key = "L5_early"
a_sat_info = satellit_info[a_sat_key]
satelliteChoice = a_sat_info[0]
start_year = a_sat_info[1]
end_year = a_sat_info[2]

a_year = 1993
a_SFblocks_key = list(SFblocks.keys())[0]


curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key])
st_date = str(a_year) + '-01-01'
ed_date = str(a_year+1) + '-01-01'
imageC = gpc.extract_satellite_IC(big_rectangle_featC = big_rectangle,
                                  start_date = st_date, end_date = ed_date,
                                  dataSource = satelliteChoice);
print (f"{imageC.size().getInfo()=}")

imageC.size().getInfo()=473
CPU times: user 56.9 ms, sys: 1.98 ms, total: 58.9 ms
Wall time: 461 ms


In [ ]:
reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
print(f"{a_sat_key = }, {a_year = }, {a_SFblocks_key = }", f"{reduced.size().getInfo()=}")

a_sat_key = 'L5_early', a_year = 1993, a_SFblocks_key = 0 reduced.size().getInfo()=1640


In [ ]:
try:
  needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
  reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
  reduced = reduced[needed_columns]

  EVI = reduced[["ID", "EVI", "system_start_time"]]
  NDVI = reduced[["ID", "NDVI", "system_start_time"]]

  EVI.dropna(subset=["EVI"], inplace=True)
  NDVI.dropna(subset=["NDVI"], inplace=True)

  outfile_name = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_) + "_EVI.csv"
  EVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

  outfile_name = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_) + "_NDVI.csv"
  NDVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)
except:
  # if counter%10==0:
  print(f"ddd, {a_sat_key = }, {a_year = }, {a_SFblocks_key = }")

In [ ]:
# block size of 1000 created this problem!!!!!
reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
print (reduced.size().getInfo())

1640


In [ ]:
%%time
needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
reduced = reduced[needed_columns]
reduced

CPU times: user 236 ms, sys: 24.1 ms, total: 260 ms
Wall time: 27.1 s


,ID,EVI,NDVI,system_start_time
0,i1,NaN,NaN,NaN
1,i2,NaN,NaN,NaN
2,i3,NaN,NaN,NaN
3,i4,NaN,NaN,NaN
4,i5,NaN,NaN,NaN
...,...,...,...,...
1635,i6,NaN,NaN,NaN
1636,i7,NaN,NaN,NaN
1637,i8,NaN,NaN,NaN
1638,i9,NaN,NaN,NaN


In [ ]:
satellit_info.keys()

dict_keys(['L4', 'L5_early', 'L5_late', 'L7'])

In [ ]:
%%time
print (DTDT.now())
bad_counter=0
done_counter = 0
problems_L5_late = dict()
for a_sat_key in ["L5_late"]:
    a_sat_info = satellit_info[a_sat_key]
    satelliteChoice = a_sat_info[0]
    start_year = a_sat_info[1]
    end_year = a_sat_info[2]
    # for a_year in range(start_year, end_year+1):
    for a_year in reversed(range(start_year, end_year+1)):
        for a_SFblocks_key in list(SFblocks.keys()):
            curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key])
            st_date = str(a_year) + '-01-01'
            ed_date = str(a_year+1) + '-01-01'
            imageC = gpc.extract_satellite_IC(big_rectangle_featC = big_rectangle,
                                              start_date = st_date, end_date = ed_date,
                                              dataSource = satelliteChoice);

            reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
            """
              The reason I have try-except below is that
              it seems some years there is no data.
              Landsat 4 has been alive since 1982 but I am not getting
              data in 1984. I am not sure what is happening.
            """
            try:
              needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
              reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
              reduced = reduced[needed_columns]

              EVI = reduced[["ID", "EVI", "system_start_time"]]
              NDVI = reduced[["ID", "NDVI", "system_start_time"]]

              EVI.dropna(subset=["EVI"], inplace=True)
              NDVI.dropna(subset=["NDVI"], inplace=True)

              name_pref = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_)
              outfile_name = name_pref + "_EVI.csv"
              EVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

              outfile_name = name_pref + "_NDVI.csv"
              NDVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)
              done_counter += 1
              if done_counter%100==0:
                print (f"{done_counter=}")
              # print(f"{a_year = }, {a_SFblocks_key = }")
            except:
              bad_counter += 1
              if bad_counter%100==0:
                print (f"{bad_counter=}")
              problems_L5_late[a_sat_key + "_" + str(bad_counter)] = a_sat_key + "_year" + str(a_year) + "_SFblock" + str(a_SFblocks_key)
              print ("---------------------------------------------------------------------------------------------")
              print(f"ddd, {a_year = }, {a_SFblocks_key = }, {reduced.size().getInfo()=}")
              print ("---------------------------------------------------------------------------------------------")

print (DTDT.now())

In [ ]:
%%time
print (DTDT.now())
bad_counter=0
done_counter = 0
problems_L7 = dict()
for a_sat_key in ["L7"]:
    a_sat_info = satellit_info[a_sat_key]
    satelliteChoice = a_sat_info[0]
    start_year = a_sat_info[1]
    end_year = a_sat_info[2]
    # for a_year in range(start_year, end_year+1):
    for a_year in reversed(range(start_year, end_year+1)):
        for a_SFblocks_key in list(SFblocks.keys()):
            curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key])
            st_date = str(a_year) + '-01-01'
            ed_date = str(a_year+1) + '-01-01'
            imageC = gpc.extract_satellite_IC(big_rectangle_featC = big_rectangle,
                                              start_date = st_date, end_date = ed_date,
                                              dataSource = satelliteChoice);

            reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
            """
              The reason I have try-except below is that
              it seems some years there is no data.
              Landsat 4 has been alive since 1982 but I am not getting
              data in 1984. I am not sure what is happening.
            """
            try:
              needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
              reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
              reduced = reduced[needed_columns]

              EVI = reduced[["ID", "EVI", "system_start_time"]]
              NDVI = reduced[["ID", "NDVI", "system_start_time"]]

              EVI.dropna(subset=["EVI"], inplace=True)
              NDVI.dropna(subset=["NDVI"], inplace=True)

              name_pref = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_)
              outfile_name = name_pref + "_EVI.csv"
              EVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

              outfile_name = name_pref + "_NDVI.csv"
              NDVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)
              done_counter += 1
              if done_counter%100==0:
                print (f"{done_counter=}")
              # print(f"{a_year = }, {a_SFblocks_key = }")
            except:
              bad_counter += 1
              if bad_counter%100==0:
                print (f"{bad_counter=}")
              problems_L7[a_sat_key + "_" + str(bad_counter)] = a_sat_key + "_year" + str(a_year) + "_SFblock" + str(a_SFblocks_key)
              print ("---------------------------------------------------------------------------------------------")
              print(f"ddd, {a_year = }, {a_SFblocks_key = }, {reduced.size().getInfo()=}")
              print ("---------------------------------------------------------------------------------------------")

print (DTDT.now())

In [ ]:
%%time
print (DTDT.now())
bad_counter=0
done_counter = 0
problems_L4 = dict()
for a_sat_key in ["L4"]:
    a_sat_info = satellit_info[a_sat_key]
    satelliteChoice = a_sat_info[0]
    start_year = a_sat_info[1]
    end_year = a_sat_info[2]
    # for a_year in range(start_year, end_year+1):
    for a_year in reversed(range(start_year, end_year+1)):
        for a_SFblocks_key in list(SFblocks.keys()):
            curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key])
            st_date = str(a_year) + '-01-01'
            ed_date = str(a_year+1) + '-01-01'
            imageC = gpc.extract_satellite_IC(big_rectangle_featC = big_rectangle,
                                              start_date = st_date, end_date = ed_date,
                                              dataSource = satelliteChoice);

            reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
            """
              The reason I have try-except below is that
              it seems some years there is no data.
              Landsat 4 has been alive since 1982 but I am not getting
              data in 1984. I am not sure what is happening.
            """
            try:
              needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
              reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
              reduced = reduced[needed_columns]

              EVI = reduced[["ID", "EVI", "system_start_time"]]
              NDVI = reduced[["ID", "NDVI", "system_start_time"]]

              EVI.dropna(subset=["EVI"], inplace=True)
              NDVI.dropna(subset=["NDVI"], inplace=True)

              name_pref = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_)
              outfile_name = name_pref + "_EVI.csv"
              EVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

              outfile_name = name_pref + "_NDVI.csv"
              NDVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)
              done_counter += 1
              if done_counter%100==0:
                print (f"{done_counter=}")
              # print(f"{a_year = }, {a_SFblocks_key = }")
            except:
              bad_counter += 1
              if bad_counter%100==0:
                print (f"{bad_counter=}")
              problems_L4[a_sat_key + "_" + str(bad_counter)] = a_sat_key + "_year" + str(a_year) + "_SFblock" + str(a_SFblocks_key)
              print ("---------------------------------------------------------------------------------------------")
              print(f"ddd, {a_year = }, {a_SFblocks_key = }, {reduced.size().getInfo()=}")
              print ("---------------------------------------------------------------------------------------------")

print (DTDT.now())

In [ ]:
%%time
print (DTDT.now())
counter=0
for a_sat_key in ["L4"]:
    a_sat_info = satellit_info[a_sat_key]
    satelliteChoice = a_sat_info[0]
    start_year = a_sat_info[1]
    end_year = a_sat_info[2]
    # for a_year in range(start_year, end_year+1):
    for a_year in reversed(range(start_year, end_year+1)):
        for a_SFblocks_key in list(SFblocks.keys()):
            curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key])
            st_date = str(a_year) + '-01-01'
            ed_date = str(a_year+1) + '-01-01'
            imageC = gpc.extract_satellite_IC(big_rectangle_featC = big_rectangle,
                                              start_date = st_date, end_date = ed_date,
                                              dataSource = satelliteChoice);

            reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas,
                                                    st_date, ed_date)

            """
              The reason I have try-except below is that
              it seems some years there is no data.
              Landsat 4 has been alive since 1982 but I am not getting
              data in 1984. I am not sure what is happening.
            """
            try:
              needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
              reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
              reduced = reduced[needed_columns]

              EVI = reduced[["ID", "EVI", "system_start_time"]]
              NDVI = reduced[["ID", "NDVI", "system_start_time"]]

              EVI.dropna(subset=["EVI"], inplace=True)
              NDVI.dropna(subset=["NDVI"], inplace=True)

              outfile_name = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_) + "_EVI.csv"
              EVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

              outfile_name = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + "_BS" + str(block_size_) + "_NDVI.csv"
              NDVI.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)
            except:
              # if counter%10==0:
              print(f"{a_sat_key = }, {a_year = }, {a_SFblocks_key = }")
              #  counter += 1

        # print ("The size of [imageC.size().getInfo()] is [{:.0f}].".format(imageC.size().getInfo()))

print (DTDT.now())

a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 0
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 1
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 2
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 3
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 4
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 5
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 6
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 7
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 8
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 9
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 10
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 11
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 12
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 13
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 14
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 15
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 16
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 17
a_sat_key = 'L4', a_year = 1984, a_SFblocks_key = 18
a_s

In [ ]:
a_sat_key = list(satellit_info.keys())[0]
a_sat_info = satellit_info[a_sat_key]
satelliteChoice = a_sat_info[0]
start_year = a_sat_info[1]
end_year = a_sat_info[2]
print (f"{satelliteChoice=}")
print (f"{start_year=}")
print (f"{end_year=}")

a_year = start_year
st_date = str(a_year) + '-01-01'
ed_date = str(a_year+10) + '-01-01'

a_SFblocks_key = list(SFblocks.keys())[0]
curr_SF_geopandas = geemap.geopandas_to_ee(SFblocks[a_SFblocks_key][0:100])
st_date = str(a_year) + '-01-01'
ed_date = str(a_year+10) + '-01-01'
imageC = gpc.extract_satellite_IC(big_rectangle_featC=big_rectangle,
                                  start_date = st_date,
                                  end_date   = ed_date,
                                  dataSource = satelliteChoice);

reduced = gpc.mosaic_and_reduce_IC_mean(imageC, curr_SF_geopandas, st_date, ed_date)
needed_columns = ["ID", "EVI", 'NDVI', "system_start_time"]
reduced = geemap.ee_to_pandas(reduced, selectors=needed_columns)
reduced = reduced[needed_columns]

satelliteChoice='LANDSAT/LT04/C02/T1_L2'
start_year=1984
end_year=1993


In [ ]:
outfile_name = "block" + str(a_SFblocks_key) + "_" + a_sat_key + "_" + st_date + "_" + ed_date + ".csv"
reduced.to_csv(out_dir_base + "/intersection_pre2008/" + outfile_name)

In [ ]:
Map = geemap.Map(center=[ymed, xmed], zoom=7)
#Map.addLayer(WA1, {'color': 'red'}, 'Western Half')
# Map.addLayer(WA2, {'color': 'red'}, 'Eastern Half')
Map.addLayer(curr_SF_geopandas, {'color': 'red'}, 'Fields')
Map

Map(center=[47.0, -120.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…